### 1. code

In [2]:
### import
import boto3
import json

### get client
s3 = boto3.client('s3')

### function for removing json files
def s3_rmv_json(path):
    '''path 경로의 json을 지운다.'''
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket, Prefix=path)
    file_list = []
    if 'Contents' in response:
        for items in response['Contents']:
            file_list.append(items['Key'])
    for i in range(0, len(file_list)):        
        s3.delete_object(Bucket=bucket, Key=file_list[i])
    print(f"deleted files: {path}")

### function for getting json list        
def s3_get_json():
    data = s3.get_object(Bucket=bucket, Key=json_path)
    json_list = json.loads(data['Body'].read())
    return json_list

### function for creating json dictionary
def create_json_dict(json_list):
    def h_get(key): 
        '''
        코드를 줄이기 위한 h_get. 
        key값의 json 객체를 가져온다.
        json 객체가 없다면 None.
        '''
        return h_item.get(key, None)
    json_dict = {'root': [], 'hospital': []}     # json dictionary의 schema
    for item in json_list:
        for key, value in item.items():
            # "ROOT_QUERY.hospital.items() => key : value로 json_dict에 저장"
            if key == "ROOT_QUERY":
                for k, v in value.items():
                    if k == 'hospital':
                        r_dict = {
                            k : v
                        }
                json_dict['root'].append(r_dict)
            # "HospitalBase:"로 시작하는 key의 items를 추출
            elif key.startswith("HospitalBase:"):
                h_item = item[key]
                h_dict = {
                    'id' : h_get('id'),
                    'name' : h_get('name'),
                    'category' : h_get('category'),
                    'category_code' : h_get('categoryCode'),
                    'category_code_list' : h_get('categoryCodeList'),
                    'category_count' : h_get('categoryCount'),
                    'description' : h_get('description'),
                    'road_address' : h_get('roadAddress'),
                    'road' : h_get('road'),
                    'rcode' : h_get('rcode'),
                    'virtual_phone' : h_get('virtualPhone'),
                    'phone' : h_get('phone'),
                    'payment_info' : h_get('paymentInfo'),
                    'conveniences' : h_get('conveniences'),
                    'review_setting' : h_get('reviewSettings'),
                    'keywords' : h_get('keywords'),
                    'booking_business_id' : h_get('bookingBusinessId'),
                    'booking_display_name' : h_get('bookingDisplayName'),
                    'visitor_reviews_score' : h_get('visitorReviewsScore'),
                    'visitor_reviews_total' : h_get('visitorReviewsTotal'),
                    'visitor_reviews_text_review_total' : h_get('visitorReviewsTextReviewTotal'),
                    'images' : h_get('images'),
                    'homepages' : h_get('homepages'),
                    'booking_url' : h_get('naverBookingUrl'),
                    'talktalk_url' : h_get('talktalkUrl'),
                    'coordinate' : h_get('coordinate'),
                }
                json_dict['hospital'].append(h_dict)
    return json_dict 

### function for getting preprocessed json dictionary
def s3_get_json_dict():
    '''json 파일을 전처리 하고 딕셔너리로 만들어 저장한다.'''
    json_list = s3_get_json()
    json_dict = create_json_dict(json_list)
    json_output = json.dumps(json_dict, ensure_ascii=False)
    path = f"{save_path}/naverplace_meta.json"
    s3.put_object(Bucket=bucket, Key=path, Body=json_output.encode('utf-8'))
    print(f"uploaded file: {path}")

### 2. run

In [ ]:
### set paths
bucket = bucket
json_path = json_path
save_path = save_path
# rmv_path = rmv_path

In [ ]:
# ### 함수 실행 (필요할 경우 주석 해제 하여 사용)
# s3_rmv_json(rmv_path)
# s3_get_json_dict()